In [1]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [2]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [3]:
soup_corona = bs(html_corona, 'html.parser')

In [4]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [5]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
#replace items using wild card and regex to have that applied
corona_df["NewCases"].replace("\,","",regex = True, inplace = True)
corona_df["NewCases"].replace("\+","", regex = True, inplace = True)
corona_df["NewCases"] = corona_df["NewCases"].astype("float")
corona_df["Date"]= dt.datetime.today()

In [6]:
corona_df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,Reported1st case,Date
0,USA,180789,17001.0,3580.0,+439,6241.0,170968,3981.0,546.0,11.0,Jan 20,2020-03-31 15:27:03.006650
1,Italy,105792,4053.0,12428.0,+837,15729.0,77635,4023.0,1750.0,206.0,Jan 29,2020-03-31 15:27:03.006650
2,Spain,94417,6461.0,8269.0,+553,19259.0,66889,5607.0,2019.0,177.0,Jan 30,2020-03-31 15:27:03.006650
3,Germany,70985,4100.0,682.0,+37,15824.0,54479,1979.0,847.0,8.0,Jan 26,2020-03-31 15:27:03.006650
4,France,52128,7578.0,3523.0,+499,9444.0,39161,5565.0,799.0,54.0,Jan 23,2020-03-31 15:27:03.006650
...,...,...,...,...,...,...,...,...,...,...,...,...
199,Papua New Guinea,1,0.0,0.0,0,0.0,1,0.0,0.1,0.0,Mar 19,2020-03-31 15:27:03.006650
200,St. Vincent Grenadines,1,0.0,0.0,0,1.0,0,0.0,9.0,0.0,Mar 10,2020-03-31 15:27:03.006650
201,Sierra Leone,1,1.0,0.0,0,0.0,1,0.0,0.1,0.0,Mar 30,2020-03-31 15:27:03.006650
202,Timor-Leste,1,0.0,0.0,0,0.0,1,0.0,0.8,0.0,Mar 20,2020-03-31 15:27:03.006650


In [7]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)
#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)
df_used["ClosedCases"] = df_used["TotalDeaths"] +  df_used["TotalRecovered"]

df_used["ClosedDeathRate"] = df_used["TotalDeaths"]/df_used["ClosedCases"]
df_used["ClosedDeathRate"].fillna(0,inplace = True)

df_used["CurrentDeathRate"] = df_used["TotalDeaths"]/df_used["TotalCases"]
df_used["CurrentDeathRate"].fillna(0,inplace = True)

total_cases = df_used["TotalCases"].sum()
total_closed_cases = df_used["ClosedCases"].sum()

df_used["Weight"] = [x/total_closed_cases for x in df_used["ClosedCases"]]

# #checking if total weight is 1 
# validate_weight = df_used["Weight"].sum()


df_used["DeathRateXWeight"] = df_used["CurrentDeathRate"] *df_used["Weight"]
df_used["DeathRateXWeight"].sum()
df_used["notes"] = ["Last 8 columns are added by Roaya" for x in df_used["DeathRateXWeight"]  ]
df_used.sort_values(by = "DeathRateXWeight", ascending = False)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,Deaths/1M pop,Reported1st case,Date,Longitude,Latitude,ClosedCases,ClosedDeathRate,CurrentDeathRate,Weight,DeathRateXWeight,notes
1,Italy,105792,4053.0,12428.0,+837,15729.0,77635,4023.0,1750.0,206.0,Jan 29,2020-03-31 15:27:03.006650,12.070013,42.796626,28157.0,0.441382,0.117476,0.129677,0.015234,Last 8 columns are added by Roaya
175,China,81518,79.0,3305.0,+5,76052.0,2161,528.0,57.0,2.0,Jan 10,2020-03-31 15:27:03.006650,103.819074,36.561765,79357.0,0.041647,0.040543,0.365480,0.014818,Last 8 columns are added by Roaya
2,Spain,94417,6461.0,8269.0,+553,19259.0,66889,5607.0,2019.0,177.0,Jan 30,2020-03-31 15:27:03.006650,-3.647550,40.244487,27528.0,0.300385,0.087580,0.126781,0.011103,Last 8 columns are added by Roaya
5,Iran,44605,3110.0,2898.0,+141,14656.0,27051,3703.0,531.0,35.0,Feb 18,2020-03-31 15:27:03.006650,54.274070,32.575033,17554.0,0.165091,0.064970,0.080845,0.005253,Last 8 columns are added by Roaya
4,France,52128,7578.0,3523.0,+499,9444.0,39161,5565.0,799.0,54.0,Jan 23,2020-03-31 15:27:03.006650,2.618787,47.824905,12967.0,0.271690,0.067584,0.059720,0.004036,Last 8 columns are added by Roaya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,Dominica,12,0.0,0.0,0,0.0,12,0.0,167.0,0.0,Mar 21,2020-03-31 15:27:03.006650,-61.357726,15.439470,0.0,0.000000,0.000000,0.000000,0.000000,Last 8 columns are added by Roaya
142,Mongolia,12,0.0,0.0,0,2.0,10,0.0,4.0,0.0,Mar 09,2020-03-31 15:27:03.006650,103.052998,46.826815,2.0,0.000000,0.000000,0.000009,0.000000,Last 8 columns are added by Roaya
111,Isle of Man,60,11.0,0.0,0,0.0,60,0.0,706.0,0.0,Mar 18,2020-03-31 15:27:03.006650,-4.538740,54.224189,0.0,0.000000,0.000000,0.000000,0.000000,Last 8 columns are added by Roaya
89,Senegal,175,13.0,0.0,0,40.0,135,0.0,10.0,0.0,Mar 01,2020-03-31 15:27:03.006650,-14.473492,14.366242,40.0,0.000000,0.000000,0.000184,0.000000,Last 8 columns are added by Roaya


In [8]:
df_used.to_csv('data.csv')
df_used.to_html('data.html')

In [9]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [10]:
corona_json

'[{"Country":"USA","TotalCases":180789,"NewCases":17001.0,"TotalDeaths":3580.0,"NewDeaths":"+439","TotalRecovered":6241.0,"ActiveCases":170968,"CriticalCases":3981.0,"Tot\\u00a0Cases\\/1M pop":546.0,"Deaths\\/1M pop":11.0,"Reported1st case":"Jan 20","Date":1585668423006,"Longitude":-97.922211,"Latitude":39.381266,"ClosedCases":9821.0,"ClosedDeathRate":0.3645249975,"CurrentDeathRate":0.0198020897,"Weight":0.0452307593,"DeathRateXWeight":0.0008956636,"notes":"Last 8 columns are added by Roaya"},{"Country":"Italy","TotalCases":105792,"NewCases":4053.0,"TotalDeaths":12428.0,"NewDeaths":"+837","TotalRecovered":15729.0,"ActiveCases":77635,"CriticalCases":4023.0,"Tot\\u00a0Cases\\/1M pop":1750.0,"Deaths\\/1M pop":206.0,"Reported1st case":"Jan 29","Date":1585668423006,"Longitude":12.07001339,"Latitude":42.79662641,"ClosedCases":28157.0,"ClosedDeathRate":0.4413822495,"CurrentDeathRate":0.1174758016,"Weight":0.1296774758,"DeathRateXWeight":0.0152339654,"notes":"Last 8 columns are added by Roaya"